# Create Training Sets for Chemical Interpolation Test
The goal of this test is to determine whether machine learning models are able to infer the interactions between elements that are not included in the training set. We will set up two different tests, 
1. Exclude a single quaternary. Withhold all entries that contain exclusively elements from the target 
2. Exclude a certain pair interaction. Withhold all entries that contain both elements in a certain pair

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from pymatgen import Composition
from itertools import product
import numpy as np
import pandas as pd
import os
import shutil

## Read in the OQMD dataset
We want only the lowest-energy entry at each composition

In [7]:
oqmd_data = pd.read_csv(os.path.join('..', 'training-data/oqmd_all.data'), delim_whitespace=True)
print('Read %d entries'%len(oqmd_data))
oqmd_data.head()
oqmd_data.shape

Read 2 entries


(2, 2)

Make all of the energies numeric

In [5]:
for col in oqmd_data.columns:
    if col == 'comp': continue
    oqmd_data[col] = pd.to_numeric(oqmd_data[col], errors='coerce')

Eliminate entries with weird formation enthalpies

In [6]:
oqmd_data.query('delta_e > -20 and delta_e < 5', inplace=True)

UndefinedVariableError: name 'delta_e' is not defined

Generate the composition object of each entry

In [ ]:
oqmd_data['comp_obj'] = oqmd_data['comp'].apply(lambda x: Composition(x))

In [ ]:
oqmd_data['pretty_comp'] = oqmd_data['comp_obj'].apply(lambda x: x.reduced_formula)

Get only the lowest-energy entry at each composition

In [ ]:
oqmd_data.sort_values('delta_e', ascending=True, inplace=True)
oqmd_data.drop_duplicates('pretty_comp', keep='first', inplace=True)
print('Reduced dataset to %d entries'%len(oqmd_data))

## Identify the systems with large numbers of entries
We want to find a system with a large amount of testing data

In [ ]:
oqmd_data['nelems'] = oqmd_data['comp_obj'].apply(lambda x: len(x))

In [ ]:
oqmd_data['system'] = oqmd_data['comp_obj'].apply(lambda x: "-".join([y.symbol for y in x]))

Get the top-10 most frequent systems

In [ ]:
oqmd_data['system'].value_counts()[:10]

*Finding*: Mn-Na-O and Fe-Na-O are the most common ternaries. So, let's choose the Na-Fe-Mn-O quaternary as a hold-out. Also, there are a large number of Ti-O binary compounds, so let's use that one as the pairwise interaction holdout

## Exclude a Quaternary Diagram
Exclude the NaFeMnO data as the hold-out set

In [ ]:
my_system = ["Na", "Fe", "Mn", "O"]

In [ ]:
def get_test_data(elems):
    """Get the data that is in any of the phase diagrams that are subsets of a certain system
    
    Ex: For Na-Fe-O, these are Na-Fe-O, Na-Fe, Na-O, Fe-O, Na-Fe, Na, Fe, O
    
    :param elems: iterable of strs, phase diagram of interest
    :return: subset of OQMD in the constituent systems"""
    
    # Generate the constituent systems
    systems = set()
    for comb in product(*[elems,]*len(elems)):
        sys = "-".join(sorted(set(comb)))
        systems.add(sys)
    
    # Query for the data
    return oqmd_data.query(' or '.join('system=="%s"'%s for s in systems))

In [ ]:
test_set = get_test_data(my_system)
print('Gathered a test set with %d entries'%len(test_set))
test_set.sample(5)

Measure the standard deviation and MAD of test set (useful for evaluating model performance)

In [ ]:
mad = np.abs(test_set['delta_e'] - test_set['delta_e'].mean()).mean()
std = test_set['delta_e'].std()
print('MAD: {:.3f} eV/atom'.format(mad))
print('Std Dev: {:.3f} eV/atom'.format(std))

Remove these entries from the dataset at large

In [ ]:
train_set = oqmd_data.loc[oqmd_data.index.difference(test_set.index)]
print('Training set size is %d entries'%len(train_set))

### Save the data in Magpie-friendly format
We will be using Magpie to generate features

In [ ]:
def save_magpie(data, path):
    """Save a dataframe in a magpie-friendly format
    
    :param data: pd.DataFrame, data to be saved
    :param path: str, output path"""
    
    data[['comp','delta_e']].to_csv(path, index=False, sep=' ')

In [ ]:
save_magpie(test_set, os.path.join('datasets', '%s_test_set.data'%(''.join(my_system))))

In [ ]:
save_magpie(train_set, os.path.join('datasets', '%s_train_set.data'%(''.join(my_system))))

## Generate the hold-pairwise out dataset

In [ ]:
my_pair = ['Ti', 'O']

In [ ]:
def get_test_data(elems):
    """Get the data that contains all of a certain set of elements.
        
    :param elems: iterable of strs, elems to exclude
    :return: subset of OQMD in the constituent systems"""
    
    # Process the dataset
    hit = oqmd_data['system'].apply(lambda x: all([e in x.split("-") for e in elems]))
    return oqmd_data[hit]

In [ ]:
test_set = get_test_data(my_pair)
print('Gathered a test set with %d entries'%len(test_set))
test_set.sample(5)

Measure the standard deviation and MAD of test set (useful for evaluating model performance)

In [ ]:
mad = np.abs(test_set['delta_e'] - test_set['delta_e'].mean()).mean()
std = test_set['delta_e'].std()
print('MAD: {:.3f} eV/atom'.format(mad))
print('Std Dev: {:.3f} eV/atom'.format(std))

Remove these entries from the dataset at large

In [ ]:
train_set = oqmd_data.loc[oqmd_data.index.difference(test_set.index)]
print('Training set size is %d entries'%len(train_set))

Save the data

In [ ]:
save_magpie(test_set, os.path.join('datasets', '%s_test_set.data'%('-'.join(my_pair))))

In [ ]:
save_magpie(train_set, os.path.join('datasets', '%s_train_set.data'%('-'.join(my_pair))))